# Week 3 Project

### Setup and imports

In [ ]:
#!conda install -c conda-forge geopy --yes

In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [19]:
#pip install lxml

In [20]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


# ANSWER 1
### Scrape and clean neighbourhod data

In [21]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [22]:
#Remove rows with unassigned borough
df=df[df.Borough != 'Not assigned']

In [23]:
#if neighbourhood is blank then fill it with borough
df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned', df['Neighbourhood'], df['Borough'])

In [24]:
#Dedupe postcodes
df= df.drop_duplicates(subset='Postcode')

In [25]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,North York
3,M4A,North York,North York
4,M5A,Downtown Toronto,Downtown Toronto
6,M6A,North York,North York
8,M7A,Queen's Park,Not assigned


In [26]:
df.shape

(103, 3)

# Answer 2

In [27]:
geos=pd.read_csv('https://cocl.us/Geospatial_data')

In [28]:
#Merge geo information onto borough dataset
merged= pd.merge(df, geos, how='inner', left_on=['Postcode'], right_on=['Postal Code'])

In [29]:
merged.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,North York,M3A,43.753259,-79.329656
1,M4A,North York,North York,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,Downtown Toronto,M5A,43.654260,-79.360636
3,M6A,North York,North York,M6A,43.718518,-79.464763
4,M7A,Queen's Park,Not assigned,M7A,43.662301,-79.389494


In [30]:
merged.shape

(103, 6)

# Answer 3

Using foursquare AP download venues for each neighbourhood

In [31]:
CLIENT_ID = 'SV1BOQ2T0JVZAJDTVXF0HHA3XKTYQVLJX1NV0VEAATL4XDGW' # your Foursquare ID
CLIENT_SECRET = 'BAHSAANBSEWO3224TSSG4LY5ICVGE1HDLQAYDYDNJV1IBR2F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SV1BOQ2T0JVZAJDTVXF0HHA3XKTYQVLJX1NV0VEAATL4XDGW
CLIENT_SECRET:BAHSAANBSEWO3224TSSG4LY5ICVGE1HDLQAYDYDNJV1IBR2F


Function to get venues in each neighbourhood

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
#Rename neighbourhood as americans spell it without u and that's what foursqaure return and it all gets confusing
merged.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [34]:
# limit neighbourhoods to a subset so don't hit foursqaure APi limit
toronto_data = merged[merged['Borough'].str.contains('Toronto')]


In [35]:
toronto_data.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,Downtown Toronto,M5A,43.654260,-79.360636
9,M5B,Downtown Toronto,Downtown Toronto,M5B,43.657162,-79.378937
15,M5C,Downtown Toronto,Downtown Toronto,M5C,43.651494,-79.375418
19,M4E,East Toronto,East Toronto,M4E,43.676357,-79.293031
20,M5E,Downtown Toronto,Downtown Toronto,M5E,43.644771,-79.373306


In [36]:
toronto_data.shape

(38, 6)

In [37]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [38]:
print(toronto_venues.shape)
toronto_venues.head()

(1176, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


---- 
Analyse the venues

In [39]:
#make into an analysable frame
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [40]:
toronto_onehot.shape

(1176, 216)

group rows by neighborhood and by taking the mean of frequency of occurrence for each category

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,Central Toronto,0.008333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.008333,0.000000,0.000000,0.016667,0.000000,0.008333,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,0.000000,0.025000,0.000000,0.075000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.008333,0.000000,0.000000,0.00000,0.000000,0.008333,0.000000,0.000000,0.033333,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.008333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008333,0.

In [42]:
toronto_grouped.shape

(4, 216)

check most common venue per neighbourhood

In [43]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.06
2     Pizza Place  0.06
3            Café  0.05
4            Park  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2   Restaurant  0.03
3        Hotel  0.03
4       Bakery  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3                Café  0.04
4             Brewery  0.03


----West Toronto----
                venue  freq
0                Café  0.06
1                 Bar  0.06
2         Coffee Shop  0.05
3  Italian Restaurant  0.04
4              Bakery  0.03




get into a dataframe

function to sort the venues in descending order.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create dataframe and display the top 10 venues per neighborhood.

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place
3,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Park,Diner,Pizza Place


<a id='item4'></a>

### Cluster the neighbourhoods

In [47]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1], dtype=int32)

New dataframe with the cluster and top 10 venues for each neighbourhood

In [48]:
toronto_data.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,Downtown Toronto,M5A,43.654260,-79.360636
9,M5B,Downtown Toronto,Downtown Toronto,M5B,43.657162,-79.378937
15,M5C,Downtown Toronto,Downtown Toronto,M5C,43.651494,-79.375418
19,M4E,East Toronto,East Toronto,M4E,43.676357,-79.293031
20,M5E,Downtown Toronto,Downtown Toronto,M5E,43.644771,-79.373306


In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,Downtown Toronto,M5A,43.654260,-79.360636,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,Downtown Toronto,M5B,43.657162,-79.378937,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
15,M5C,Downtown Toronto,Downtown Toronto,M5C,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
19,M4E,East Toronto,East Toronto,M4E,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place
20,M5E,Downtown Toronto,Downtown Toronto,M5E,43.644771,-79.373306,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant


Map the resulting clusters

In [68]:
#Get the lat and long of Tornto to centre map
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [69]:
latitude = 43.6532 
longitude = -79.3832 

In [70]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

Examine Clusters and assign a name

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,-79.388790,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
62,Central Toronto,-79.416936,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
67,Central Toronto,-79.390197,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
68,Central Toronto,-79.411307,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
73,Central Toronto,-79.405678,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
74,Central Toronto,-79.405678,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
79,Central Toronto,-79.388790,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
83,Central Toronto,-79.383160,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym
86,Central Toronto,-79.400049,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Restaurant,Dessert Shop,Sushi Restaurant,Clothing Store,Gym


In [ ]:
# name = North: Coffee shops and sandwiches

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,-79.360636,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
9,Downtown Toronto,-79.378937,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
15,Downtown Toronto,-79.375418,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
20,Downtown Toronto,-79.373306,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
24,Downtown Toronto,-79.387383,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
25,Downtown Toronto,-79.422564,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
30,Downtown Toronto,-79.384568,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
31,West Toronto,-79.442259,1,Bar,Café,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Park,Diner,Pizza Place
36,Downtown Toronto,-79.381752,1,Coffee Shop,Café,Restaurant,Bakery,Hotel,Gastropub,Park,Steakhouse,Japanese Restaurant,Italian Restaurant
37,West Toronto,-79.419750,1,Bar,Café,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Park,Diner,Pizza Place


In [ ]:
#name= South, like coffee shops and cafes

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,-79.293031,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place
41,East Toronto,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place
47,East Toronto,-79.315572,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place
54,East Toronto,-79.340923,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place
100,East Toronto,-79.321558,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Park,Brewery,Ice Cream Shop,Yoga Studio,Pub,Pizza Place


In [ ]:
#name=  East, likes Greek restaurants